## robot server

In [64]:
#!/usr/bin/env python
#example
import grpc
import rospy
from concurrent import futures
# import the publisher and subscriber node
from ros_bridge import RosBridge
# import the gRPC generated classes
from robo_gym_server_modules.robot_server.grpc_msgs.python import robot_server_pb2, robot_server_pb2_grpc

class Client():

    def __init__(self, server_port):
        self.channel = grpc.insecure_channel('[::]:' + repr(server_port))
        self.robot_server_stub = robot_server_pb2_grpc.RobotServerStub(self.channel)

    def set_state(self, state):
        # Old method, to be gradually replaced in all the stack
        msg = self.robot_server_stub.SetState(robot_server_pb2.State(state = state), timeout = 60)
        return msg.success

    def set_state_msg(self, state_msg):
        msg = self.robot_server_stub.SetState(state_msg, timeout = 60)
        return msg.success

    def get_state(self,):
        # Old method, to be gradually replaced in all the stack
        msg = self.robot_server_stub.GetState(robot_server_pb2.Empty(), timeout = 20)
        if msg.success == 1:
            return msg.state
        else:
            raise Exception('Error while retrieving state')

    def get_state_msg(self,):
        msg = self.robot_server_stub.GetState(robot_server_pb2.Empty(), timeout = 20)
        if msg.success == 1:
            return msg
        else:
            raise Exception('Error while retrieving state')

    def send_action(self, action):
        msg = self.robot_server_stub.SendAction(robot_server_pb2.Action(action = action), timeout = 20 )
        return msg.success

    def send_action_get_state(self, action):
        msg = self.robot_server_stub.SendActionGetState(robot_server_pb2.Action(action = action), timeout = 20 )
        if msg.success == 1:
            return msg
        else:
            raise Exception('Error while retrieving state')


## init robot server

In [65]:
rospy.loginfo('Starting Example Robot Server...')
server_port = 50051
client = Client(server_port)
channel = grpc.insecure_channel('[::]:' + repr(server_port))
robot_server_stub = robot_server_pb2_grpc.RobotServerStub(channel)

## get state

In [3]:
state = client.get_state()
print("state: ", state)

state:  [-0.13905775547027588, -0.4946856200695038, -0.6523081064224243, 0.9823417067527771, 0.004937502555549145, -0.009113935753703117, -0.0040998985059559345, -0.007414854597300291, 2.2102065599938214e-07, 9.38727964694408e-07, 1.2137742260165396e-06, -0.005454658064991236, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [8]:
get_state_msg =client.get_state_msg()
# print(get_state_msg)
print("state: ",get_state_msg.state)
print("float_params: ", get_state_msg.float_params)
print("string_params: ", get_state_msg.string_params)
print("state_dict: ", get_state_msg.state_dict)
state = get_state_msg.state
print(len(state))



state:  [0.15608561038970947, -0.705842137336731, -1.0894068479537964, 0.7519558668136597, 0.0032182764261960983, 0.0006526880315504968, 0.002745655830949545, -0.0019451521802693605, 2.2424846974899992e-06, -0.0006543011404573917, -0.001473238691687584, -3.6879755498375744e-05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
float_params:  {}
string_params:  {}
state_dict:  {'rear_left_wheel_joint_velocity': -0.001473238691687584, 'front_left_steering_joint_velocity_ref': 0.0, 'front_right_steering_joint_velocity': 0.0006526880315504968, 'rear_right_steering_joint_velocity': -0.0019451521802693605, 'front_right_steering_joint_velocity_ref': 0.0, 'rear_right_steering_joint_position': 0.7519558668136597, 'front_left_wheel_joint_velocity': 2.2424846974899992e-06, 'front_right_steering_joint_position': -0.705842137336731, 'front_right_steering_joint_position_ref': 0.0, 'front_right_wheel_joint_velocity': -0.0006543011404573917, 'rear_left_steering_joint_velocity_ref': 0.0, 'front_left_steering_joi

## set state

In [146]:
import numpy as np
link_names = [  'front_left_steering_link', 
                'front_right_steering_link', 
                'rear_left_steering_link', 
                'rear_right_steering_link'] 
link_state = dict.fromkeys(link_names, np.deg2rad(90))
link_state.update({'reset_links': False})
state_dict_msg=link_state
print(state_dict_msg)
# print(link_state)
joint_pos_ref = [1,2,3,4]
joint_vel_ref = [0,0,0,0]
state_msg = joint_pos_ref + joint_vel_ref
print(state_msg)
print(state_msg[:4])
print(state_msg[4:8])


{'front_left_steering_link': 1.5707963267948966, 'front_right_steering_link': 1.5707963267948966, 'rear_left_steering_link': 1.5707963267948966, 'rear_right_steering_link': 1.5707963267948966, 'reset_links': False}
[1, 2, 3, 4, 0, 0, 0, 0]
[1, 2, 3, 4]
[0, 0, 0, 0]


In [147]:

# state_msg = robot_server_pb2.State(state_dict = client.get_state_msg().state_dict)
# link_state = {'rear_right_steering_link': 0}
link_state_msg = robot_server_pb2.State(state=state_msg, state_dict=state_dict_msg)
print(link_state_msg)
client.set_state_msg(link_state_msg)


state: 1.0
state: 2.0
state: 3.0
state: 4.0
state: 0.0
state: 0.0
state: 0.0
state: 0.0
state_dict {
  key: "front_left_steering_link"
  value: 1.5707963705062866
}
state_dict {
  key: "front_right_steering_link"
  value: 1.5707963705062866
}
state_dict {
  key: "rear_left_steering_link"
  value: 1.5707963705062866
}
state_dict {
  key: "rear_right_steering_link"
  value: 1.5707963705062866
}
state_dict {
  key: "reset_links"
  value: 0.0
}



True

## send action

In [67]:
import time
class Robot:
    def __init__(self):
        self.get_parameters()

    def get_parameters(self):
        self.limit_upper = rospy.get_param('/limit_upper', +120.)
        self.limit_lower = rospy.get_param('/limit_lower', -120.)

        action_cicle_rate = rospy.get_param('/action_cycle_rate', 25.)
        robot_cicle_rate = rospy.get_param('/robot_actuation_cycle_rate', action_cicle_rate)
        action_generation_time = rospy.get_param('/action_generation_time', 0.01)
        action_cicle_time = 1/action_cicle_rate
        robot_cicle_time = 1/robot_cicle_rate
        sleep_time = action_cicle_time - action_generation_time
        
        self.ratio= int(robot_cicle_rate/action_cicle_rate)
        self.ac_rate = action_cicle_rate
        self.rc_rate = robot_cicle_rate
        self.control_period = sleep_time

        # iinit finish msg
        print(f"============Config============")
        print(f"ac_rate: {action_cicle_rate}, rc_rate: {robot_cicle_rate}, ratio: {self.ratio}")
        print(f"ac time: {action_cicle_time}, rc_time: {robot_cicle_time}")
        print(f"sleep_time: {sleep_time}, action_generation_time {action_generation_time}")
robot = Robot()

============Config============
ac_rate: 10, rc_rate: 10, ratio: 1
ac time: 0.1, rc_time: 0.1
sleep_time: 0.095, action_generation_time 0.005


In [72]:
for i in range(10):
    action = [1000, 1000, 1000, 1000]
    print(action)
    print(type(action))
    client.send_action(action)
    time.sleep(robot.control_period)

[1000, 1000, 1000, 1000]
<class 'list'>
[1000, 1000, 1000, 1000]
<class 'list'>
[1000, 1000, 1000, 1000]
<class 'list'>
[1000, 1000, 1000, 1000]
<class 'list'>
[1000, 1000, 1000, 1000]
<class 'list'>
[1000, 1000, 1000, 1000]
<class 'list'>
[1000, 1000, 1000, 1000]
<class 'list'>
[1000, 1000, 1000, 1000]
<class 'list'>
[1000, 1000, 1000, 1000]
<class 'list'>
[1000, 1000, 1000, 1000]
<class 'list'>


In [92]:
for i in range(10):
    action = [100, 1000, 1000, 1000]
    # print(action)
    obs = client.send_action_get_state(action)
    print(obs.state)
    time.sleep(robot.control_period)

[0.38733741641044617, 2.3213443756103516, 2.321316957473755, 2.3212902545928955, 0.029848612844944, 0.06066366657614708, -0.015620495192706585, -0.002110446337610483, 0.004526871256530285, 0.007080720271915197, -0.00019912012794520706, 0.0015211118152365088, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.3895858824253082, 2.321307897567749, 2.321293592453003, 2.321291446685791, 0.018948553130030632, -0.0038503408432006836, -0.0008845195989124477, -0.00042072663200087845, 0.001492965966463089, -0.0004924186505377293, -0.0001558042858960107, 0.00039220519829541445, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.3920835554599762, 2.3212907314300537, 2.32129168510437, 2.3212931156158447, 0.016405995935201645, 0.00013287740875966847, -0.0004677175311371684, -0.0007667429745197296, -0.0014384984970092773, -0.0002551387879066169, -0.00027085409965366125, 0.00033367794821970165, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.3941478431224823, 2.3213412761688232, 2.3213138580322266, 2.321293592453003, 0.02

## contrler PID

In [95]:





        
robot = Robot()
robot.print_param()

limit_upper: 2.321287905152458
limit_lower: -2.321287905152458
limit_effort: 1000
limit_velocity: 10
============Config============
ac_rate: 10, rc_rate: 10, ratio: 1
ac time: 0.1, rc_time: 0.1
sleep_time: 0.095, action_generation_time 0.005


In [141]:
class Controller:
    def __init__(self, K=[1, 0, 0], sp=0, max=None):
        self.kp, self.ki, self.kd= K
        self.sp= sp
        self.last_error= 0
        self.last_time= time.time()
        self.eint= 0
        self.dt = 0 
        self.max =  max

    def get_action(self, angle, velocity, sp=None):
        current_time = time.time()
        if sp:
            self.sp = sp
        self.dt = current_time - self.last_time
        self.pv = angle
        error = self.sp - self.pv
        self.eint += error * self.dt
        self.edot =  (error - self.last_error) / self.dt
        self.cv = self.kp * error + self.ki * self.eint + self.kd * self.edot
        cv = np.clip(self.cv,-self.max,self.max)
        self.last_error= error
        self.last_time= current_time
        return cv

In [138]:
import copy
import time

class AgribotEnv:
    def __init__(self):
        self.get_parameters()
        self.client = Client(50051)
        self.link_names = [ 'front_left_steering_link', 
                            'front_right_steering_link', 
                            'rear_left_steering_link', 
                            'rear_right_steering_link'] 
        self.joint_names = ['front_left_steering_joint', 
                            'front_right_steering_joint', 
                            'rear_left_steering_joint', 
                            'rear_right_steering_joint', 
                            'front_left_wheel_joint', 
                            'front_right_wheel_joint', 
                            'rear_left_wheel_joint', 
                            'rear_right_wheel_joint']

    def get_robot_server_composition(self) -> list:
        rs_state_keys = [
            'front_left_steering_joint_position',
            'front_right_steering_joint_position',
            'rear_left_steering_joint_position',
            'rear_right_steering_joint_position',

            'front_left_steering_joint_velocity',
            'front_right_steering_joint_velocity',
            'rear_left_steering_joint_velocity',
            'rear_right_steering_joint_velocity',
            'front_left_wheel_joint_velocity',
            'front_right_wheel_joint_velocity',
            'rear_left_wheel_joint_velocity',
            'rear_right_wheel_joint_velocity',

            'front_left_steering_joint_position_ref',
            'front_right_steering_joint_position_ref',
            'rear_left_steering_joint_position_ref',
            'rear_right_steering_joint_position_ref',
            
            'front_left_steering_joint_velocity_ref',
            'front_right_steering_joint_velocity_ref',
            'rear_left_steering_joint_velocity_ref',
            'rear_right_steering_joint_velocity_ref'
        ]
        return rs_state_keys

    def _check_rs_state_keys(self, rs_state) -> None:
        keys = self.get_robot_server_composition()
        if not len(keys) == len(rs_state.keys()):
            print("Robot Server state keys to not match. Different lengths.")

        
        for key in keys:
            if key not in rs_state.keys():
                print("Robot Server state keys to not match")


    def get_state(self):
        state_msg = client.get_state_msg()
        state = state_msg.state
        state_dict = state_msg.state_dict
        self.joint_position = copy.deepcopy(state[:4])
        self.joint_velocity = copy.deepcopy(state[4:8])
        self.joint_position_ref = copy.deepcopy(state[12:16])
        self.joint_velocity_ref = copy.deepcopy(state[16:20])
        return state_dict
    
    def set_state(self,joint_pos_ref=[0]*4,joint_vel_ref=[0]*4, state_dict={}):
        state_msg = joint_pos_ref + joint_vel_ref
        state_dict_msg = state_dict
        link_state_msg = robot_server_pb2.State(state=state_msg, state_dict=state_dict_msg)
        client.set_state_msg(link_state_msg)

    def send_action(self, action=[0]*4):
        self.client.send_action(action)
    
    def reset(self):
        pass

    def step(self, action=[0]*4):
        if not len(action) == 4:
            print("not 4 actions")

        rs_state = self.client.send_action_get_state(action).state_dict
        self._check_rs_state_keys(rs_state)
        return rs_state

    def get_parameters(self):
        self.limit_upper = rospy.get_param('/limit_upper', +120.)
        self.limit_lower = rospy.get_param('/limit_lower', -120.)
        self.limit_effort = rospy.get_param('/limit_effort', 1000.)
        self.limit_velocity = rospy.get_param('/limit_velocity', 10.)

        self.action_cicle_rate = rospy.get_param('/action_cycle_rate', 25.)
        self.robot_cicle_rate = rospy.get_param('/robot_actuation_cycle_rate', self.action_cicle_rate)
        self.action_generation_time = rospy.get_param('/action_generation_time', 0.01)
        self.action_cicle_time = 1/self.action_cicle_rate
        self.robot_cicle_time = 1/self.robot_cicle_rate
        self.sleep_time = self.action_cicle_time - self.action_generation_time
        
        self.ratio= int(self.robot_cicle_rate/self.action_cicle_rate)
        self.ac_rate = self.action_cicle_rate
        self.rc_rate = self.robot_cicle_rate
        self.control_period = self.sleep_time
        
    def print_param(self):
        print(f"limit_upper: {self.limit_upper}")
        print(f"limit_lower: {self.limit_lower}")
        print(f"limit_effort: {self.limit_effort}")
        print(f"limit_velocity: {self.limit_velocity}")
        # iinit finish msg
        print(f"============Config============")
        print(f"ac_rate: {self.action_cicle_rate}, rc_rate: {self.robot_cicle_rate}, ratio: {self.ratio}")
        print(f"ac time: {self.action_cicle_time}, rc_time: {self.robot_cicle_time}")
        print(f"sleep_time: {self.sleep_time}, action_generation_time {self.action_generation_time}")

agribot = AgribotEnv()
agribot.print_param()

joint_pos_ref = [1,2,-1,-2]
# joint_vel_ref = [0,1,0,1]
# agribot.set_state(joint_pos_ref,joint_vel_ref)
agribot.set_state()
action = [-1000, -1000, -1000, -1000]
for _ in range(10):
    agribot.send_action(action)
agribot.send_action()
obs = agribot.get_state()
print(agribot.joint_position)
print(agribot.joint_velocity)
print(agribot.joint_position_ref)
print(agribot.joint_velocity_ref)
print(obs)

limit_upper: 2.321287905152458
limit_lower: -2.321287905152458
limit_effort: 1000
limit_velocity: 10
============Config============
ac_rate: 10, rc_rate: 10, ratio: 1
ac time: 0.1, rc_time: 0.1
sleep_time: 0.095, action_generation_time 0.005
[-2.3212907314300537, -2.3212904930114746, -2.32128643989563, -2.3212900161743164]
[-5.6859906180761755e-05, 0.00019053483265452087, -0.0026990449987351894, 6.570164259755984e-05]
[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
{'rear_left_steering_joint_velocity_ref': 0.0, 'rear_right_steering_joint_position': -2.3212900161743164, 'rear_left_steering_joint_velocity': -0.0026990449987351894, 'front_right_steering_joint_position': -2.3212904930114746, 'front_right_steering_joint_velocity_ref': 0.0, 'front_left_steering_joint_velocity': -5.6859906180761755e-05, 'front_left_wheel_joint_velocity': -0.00012720040103886276, 'front_right_wheel_joint_velocity': -0.00034707417944446206, 'front_left_steering_joint_position_ref': 0.0, 'front_left_steering_joint_vel

In [144]:
import math
import time
import numpy as np

obs = agribot.get_state()
K = [2000, 100, 500] #no_fmu
# K = [5, 0.1, 5]
controller = [
    Controller(K, sp=obs['front_left_steering_joint_position_ref'], max=agribot.limit_effort), 
    Controller(K, sp=obs['front_right_steering_joint_position_ref'], max=agribot.limit_effort), 
    Controller(K, sp=obs['rear_left_steering_joint_position_ref'], max=agribot.limit_effort), 
    Controller(K, sp=obs['rear_right_steering_joint_position_ref'], max=agribot.limit_effort)]

action = [0]*4
Done = True
it = 0
tol = np.deg2rad(1)
time.sleep(agribot.control_period)

while Done:
    action[0] = controller[0].get_action(angle=obs['front_left_steering_joint_position'], velocity=obs['front_left_steering_joint_velocity'], sp=obs['front_left_steering_joint_position_ref'])
    action[1] = controller[1].get_action(angle=obs['front_right_steering_joint_position'], velocity=obs['front_right_steering_joint_velocity'], sp=obs['front_right_steering_joint_position_ref'])
    action[2] = controller[2].get_action(angle=obs['rear_left_steering_joint_position'], velocity=obs['rear_left_steering_joint_velocity'], sp=obs['rear_left_steering_joint_position_ref'])
    action[3] = controller[3].get_action(angle=obs['rear_right_steering_joint_position'], velocity=obs['rear_right_steering_joint_velocity'], sp=obs['rear_right_steering_joint_position_ref'])
    obs = agribot.step(action)
    time.sleep(robot.control_period)
    for i in range(4):
        print(f"[{i}]:: obs:{controller[i].pv:.2f} act:{controller[i].cv:.2f} dt:{controller[i].dt:.3f} error:{controller[i].last_error:.3f}| sp:{controller[i].sp:.2f}, eint: {controller[i].eint:.2e}| ")

    if abs(controller[0].last_error) < tol and abs(obs['front_left_steering_joint_velocity']) < 0.1 and \
        abs(controller[1].last_error) < tol and abs(obs['front_right_steering_joint_velocity']) < 0.1 and \
        abs(controller[2].last_error) < tol and abs(obs['rear_left_steering_joint_velocity']) < 0.1 and \
        abs(controller[3].last_error) < tol and abs(obs['rear_right_steering_joint_velocity']) < 0.1:
        # client.set_state(get_random_state())
        # obs = client.get_state()
        # controller = Controller(K, obs[2], max_action)
        Done=False
    it +=1
    if it ==1000:
        Done=False

[0]:: obs:1.99 act:-14335.41 dt:0.096 error:-1.990| sp:0.00, eint: -1.92e-01| 
[1]:: obs:0.99 act:-7136.61 dt:0.096 error:-0.992| sp:0.00, eint: -9.56e-02| 
[2]:: obs:1.02 act:-7343.53 dt:0.096 error:-1.021| sp:0.00, eint: -9.84e-02| 
[3]:: obs:-1.02 act:7338.60 dt:0.096 error:1.020| sp:0.00, eint: 9.84e-02| 
[0]:: obs:1.84 act:-3317.43 dt:0.190 error:-1.835| sp:0.00, eint: -5.40e-01| 
[1]:: obs:0.89 act:-1532.55 dt:0.190 error:-0.889| sp:0.00, eint: -2.64e-01| 
[2]:: obs:0.87 act:-1353.77 dt:0.190 error:-0.867| sp:0.00, eint: -2.63e-01| 
[3]:: obs:-0.98 act:1870.92 dt:0.190 error:0.977| sp:0.00, eint: 2.84e-01| 
[0]:: obs:1.60 act:-2658.63 dt:0.192 error:-1.597| sp:0.00, eint: -8.47e-01| 
[1]:: obs:0.68 act:-846.16 dt:0.192 error:-0.678| sp:0.00, eint: -3.95e-01| 
[2]:: obs:0.67 act:-866.05 dt:0.192 error:-0.670| sp:0.00, eint: -3.92e-01| 
[3]:: obs:-0.91 act:1685.57 dt:0.192 error:0.909| sp:0.00, eint: 4.59e-01| 
[0]:: obs:1.36 act:-2194.40 dt:0.191 error:-1.357| sp:0.00, eint: -1.11

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "{"created":"@1646338906.406531729","description":"Error received from peer ipv6:[::]:50051","file":"src/core/lib/surface/call.cc","file_line":1061,"grpc_message":"Socket closed","grpc_status":14}"
>

In [109]:
import math
import time
import numpy as np

agribot.get_state()

# print(obs)
sp = np.deg2rad(0)
max_action = robot.limit_effort
K = [2000, 100, 500] #no_fmu
# K = [5, 0.1, 5]
controller = [Controller(K, sp, max_action), Controller(K, sp, max_action), Controller(K, sp, max_action), Controller(K, sp, max_action)]
print(controller)
# # time_check.reset()
action = [0]*4
Done = True
it = 0
tol = np.deg2rad(1)
print(tol)
time.sleep(robot.control_period)

while Done:
    action[0] = controller[0].get_action(angle=obs['front_left_steering_joint_position'], velocity=obs['front_left_steering_joint_velocity'])
    action[1] = controller[1].get_action(angle=obs['front_right_steering_joint_position'], velocity=obs['front_right_steering_joint_velocity'])
    action[2] = controller[2].get_action(angle=obs['rear_left_steering_joint_position'], velocity=obs['rear_left_steering_joint_velocity'])
    action[3] = controller[3].get_action(angle=obs['rear_right_steering_joint_position'], velocity=obs['rear_right_steering_joint_velocity'])
    obs = client.send_action_get_state(action).state_dict
    time.sleep(robot.control_period)
    for i in range(4):
        print(f"[{i}]:: obs:{controller[i].pv:.2f} act:{controller[i].cv:.2f} dt:{controller[i].dt:.3f} error:{controller[i].last_error:.3f}| sp:{controller[i].sp:.2f}, eint: {controller[i].eint:.2e}| ")

    if abs(controller[0].last_error) < tol and abs(obs['front_left_steering_joint_velocity']) < 0.1 and \
        abs(controller[1].last_error) < tol and abs(obs['front_right_steering_joint_velocity']) < 0.1 and \
        abs(controller[2].last_error) < tol and abs(obs['rear_left_steering_joint_velocity']) < 0.1 and \
        abs(controller[3].last_error) < tol and abs(obs['rear_right_steering_joint_velocity']) < 0.1:
        # client.set_state(get_random_state())
        # obs = client.get_state()
        # controller = Controller(K, obs[2], max_action)
        Done=False
    it +=1
    if it ==1000:
        Done=False

In [ ]:
from sensor_msgs.msg import JointState
joint_names = [ 'front_left_steering_joint', 
                'front_right_steering_joint', 
                'rear_left_steering_joint', 
                'rear_right_steering_joint', 
                'front_left_wheel_joint', 
                'front_right_wheel_joint', 
                'rear_left_wheel_joint', 
                'rear_right_wheel_joint']
action = [1,2,3,4]
env_cmd_msg = JointState()
env_cmd_msg.name = joint_names[:4]
env_cmd_msg.effort = action
print(env_cmd_msg.name)
print(env_cmd_msg.effort)
d = {'a':1, 'b':2}
for i in d:
    print(i)
